
# Architectural Style Generation

In [1]:
__author__ = "Caroline Ho and Cole Thomson"
__version__ = "CS230, Stanford, Autumn 2018 term"

## Contents
1. [Overview](#Overview)
2. [Set-Up](#Set-Up)
3. [Model](#Model)
5. [Resources](#Resources)

## Overview

In this notebook, we use conditional GANs to generate images of buildings with given architectural styles.

## Set-Up

Run the following cells to import necessary libraries/functions and set modes.

In [2]:
from __future__ import print_function
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torchvision.utils as vutils

We now set constants.

In [3]:
nz = 100
ngf = 64
ndf = 64
nc = 3
epochs = 25
lr = 0.0002
beta1 = 0.5
ngpu = 1
netG_path, netD_path = '', ''

random.seed(0)
torch.manual_seed(0)

cudnn.benchmark = True

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

We create a folder to store our generated images.

In [4]:
generated_images = 'generated_images'

try: os.makedirs(generated_images)
except OSError: pass

### Data

Download [Xu et al.'s architecture dataset](https://drive.google.com/file/d/0Bwo0SFiZwl3JVGRlWGZUaW5va00/edit) and place it in the current directory.

This dataset contains 4,843 images of buildings from 25 architecture style classes ranging from Achaemenid to Tudor Revival. Image dimensions/aspect ratios are not consistent.

In [5]:
transform = transforms.Compose([
    transforms.Resize(64),
    transforms.CenterCrop(64),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

dataset = datasets.ImageFolder('arcDataset', transform=transform)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=64, shuffle=True, num_workers=4)

## Model

In [6]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

### Generator

In [7]:
class Generator(nn.Module):
    def __init__(self, ngpu):
        super(Generator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            # input is Z, going into a convolution
            nn.ConvTranspose2d(nz, ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            # state size. (ngf*8) x 4 x 4
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            # state size. (ngf*4) x 8 x 8
            nn.ConvTranspose2d(ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            # state size. (ngf*2) x 16 x 16
            nn.ConvTranspose2d(ngf * 2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            # state size. (ngf) x 32 x 32
            nn.ConvTranspose2d(ngf, nc, 4, 2, 1, bias=False),
            nn.Tanh()
            # state size. (nc) x 64 x 64
        )

    def forward(self, input):
        if input.is_cuda and self.ngpu > 1:
            output = nn.parallel.data_parallel(self.main, input, range(self.ngpu))
        else:
            output = self.main(input)
        return output

In [8]:
netG = Generator(ngpu).to(device)
netG.apply(weights_init)
if netG_path != '':
    netG.load_state_dict(torch.load(netG_path))
print(netG)

Generator(
  (main): Sequential(
    (0): ConvTranspose2d(100, 512, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
    (3): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace)
    (6): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace)
    (9): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU(inplace)
    (12): ConvTranspose2d(64, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (13): Tanh()
  )
)


### Discriminator

In [9]:
class Discriminator(nn.Module):
    def __init__(self, ngpu):
        super(Discriminator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            # input is (nc) x 64 x 64
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf) x 32 x 32
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*2) x 16 x 16
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*4) x 8 x 8
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*8) x 4 x 4
            nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input):
        if input.is_cuda and self.ngpu > 1:
            output = nn.parallel.data_parallel(self.main, input, range(self.ngpu))
        else:
            output = self.main(input)

        return output.view(-1, 1).squeeze(1)

In [10]:
netD = Discriminator(ngpu).to(device)
netD.apply(weights_init)
if netD_path != '':
    netD.load_state_dict(torch.load(netD_path))
print(netD)

Discriminator(
  (main): Sequential(
    (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): LeakyReLU(negative_slope=0.2, inplace)
    (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): LeakyReLU(negative_slope=0.2, inplace)
    (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): LeakyReLU(negative_slope=0.2, inplace)
    (8): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (9): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): LeakyReLU(negative_slope=0.2, inplace)
    (11): Conv2d(512, 1, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (12): Sigmoid()
  )
)


### Run DCGan

In [ ]:
criterion = nn.BCELoss()

fixed_noise = torch.randn(64, nz, 1, 1, device=device)
real_label = 1
fake_label = 0

# setup optimizer
optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999))

for epoch in range(epochs):
    for i, data in enumerate(dataloader, 0):
        ############################
        # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
        ###########################
        # train with real
        netD.zero_grad()
        real_cpu = data[0].to(device)
        batch_size = real_cpu.size(0)
        label = torch.full((batch_size,), real_label, device=device)
        
        output = netD(real_cpu)
        errD_real = criterion(output, label)
        errD_real.backward()
        D_x = output.mean().item()

        # train with fake
        noise = torch.randn(batch_size, nz, 1, 1, device=device)
        fake = netG(noise)
        label.fill_(fake_label)
        output = netD(fake.detach())
        errD_fake = criterion(output, label)
        errD_fake.backward()
        D_G_z1 = output.mean().item()
        errD = errD_real + errD_fake
        optimizerD.step()

        ############################
        # (2) Update G network: maximize log(D(G(z)))
        ###########################
        netG.zero_grad()
        label.fill_(real_label)  # fake labels are real for generator cost
        output = netD(fake)
        errG = criterion(output, label)
        errG.backward()
        D_G_z2 = output.mean().item()
        optimizerG.step()

        print('[%d/%d][%d/%d] Loss_D: %.4f Loss_G: %.4f D(x): %.4f D(G(z)): %.4f / %.4f'
              % (epoch, epochs, i, len(dataloader),
                 errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))
        if i % 100 == 0:
            vutils.save_image(real_cpu,
                    '%s/real_samples.png' % generated_images,
                    normalize=True)
            fake = netG(fixed_noise)
            vutils.save_image(fake.detach(),
                    '%s/fake_samples_epoch_%03d.png' % (generated_images, epoch),
                    normalize=True)

    # do checkpointing
    torch.save(netG.state_dict(), '%s/netG_epoch_%d.pth' % (generated_images, epoch))
    torch.save(netD.state_dict(), '%s/netD_epoch_%d.pth' % (generated_images, epoch))

[0/25][0/75] Loss_D: 1.9715 Loss_G: 1.8995 D(x): 0.2516 D(G(z)): 0.2518 / 0.1726
[0/25][1/75] Loss_D: 2.8722 Loss_G: 3.9928 D(x): 0.9482 D(G(z)): 0.9107 / 0.0335
[0/25][2/75] Loss_D: 1.2831 Loss_G: 6.2906 D(x): 0.9256 D(G(z)): 0.6264 / 0.0028
[0/25][3/75] Loss_D: 0.4160 Loss_G: 6.2165 D(x): 0.8663 D(G(z)): 0.1792 / 0.0028
[0/25][4/75] Loss_D: 0.6690 Loss_G: 5.5850 D(x): 0.8078 D(G(z)): 0.2259 / 0.0058
[0/25][5/75] Loss_D: 1.0724 Loss_G: 6.7921 D(x): 0.7805 D(G(z)): 0.4032 / 0.0016
[0/25][6/75] Loss_D: 0.5900 Loss_G: 6.6412 D(x): 0.8233 D(G(z)): 0.1438 / 0.0019
[0/25][7/75] Loss_D: 0.7179 Loss_G: 7.0333 D(x): 0.7907 D(G(z)): 0.2505 / 0.0012
[0/25][8/75] Loss_D: 0.8716 Loss_G: 8.0299 D(x): 0.7665 D(G(z)): 0.2792 / 0.0004
[0/25][9/75] Loss_D: 0.4904 Loss_G: 7.7365 D(x): 0.8248 D(G(z)): 0.1670 / 0.0006
[0/25][10/75] Loss_D: 0.7406 Loss_G: 8.4901 D(x): 0.7789 D(G(z)): 0.2396 / 0.0003
[0/25][11/75] Loss_D: 0.6399 Loss_G: 9.7865 D(x): 0.8683 D(G(z)): 0.2396 / 0.0001
[0/25][12/75] Loss_D: 0.47

[1/25][25/75] Loss_D: 0.1952 Loss_G: 8.0219 D(x): 0.8981 D(G(z)): 0.0003 / 0.0005
[1/25][26/75] Loss_D: 0.5992 Loss_G: 8.4020 D(x): 0.9313 D(G(z)): 0.3442 / 0.0003
[1/25][27/75] Loss_D: 0.2072 Loss_G: 7.9343 D(x): 0.9308 D(G(z)): 0.0684 / 0.0005
[1/25][28/75] Loss_D: 0.4482 Loss_G: 8.6766 D(x): 0.8717 D(G(z)): 0.2075 / 0.0003
[1/25][29/75] Loss_D: 0.6427 Loss_G: 6.1386 D(x): 0.7433 D(G(z)): 0.0619 / 0.0032
[1/25][30/75] Loss_D: 1.3342 Loss_G: 16.5938 D(x): 0.8390 D(G(z)): 0.5769 / 0.0000
[1/25][31/75] Loss_D: 2.1889 Loss_G: 12.7911 D(x): 0.3505 D(G(z)): 0.0000 / 0.0000
[1/25][32/75] Loss_D: 0.1790 Loss_G: 7.6190 D(x): 0.8751 D(G(z)): 0.0008 / 0.0012
[1/25][33/75] Loss_D: 0.3440 Loss_G: 5.8809 D(x): 0.9305 D(G(z)): 0.1471 / 0.0044
[1/25][34/75] Loss_D: 0.8636 Loss_G: 16.2703 D(x): 0.9787 D(G(z)): 0.5189 / 0.0000
[1/25][35/75] Loss_D: 0.3521 Loss_G: 16.6253 D(x): 0.7918 D(G(z)): 0.0000 / 0.0000
[1/25][36/75] Loss_D: 0.8100 Loss_G: 8.7260 D(x): 0.6617 D(G(z)): 0.0008 / 0.0025
[1/25][37/75

[2/25][50/75] Loss_D: 0.7308 Loss_G: 3.5942 D(x): 0.7654 D(G(z)): 0.2905 / 0.0474
[2/25][51/75] Loss_D: 0.7312 Loss_G: 3.1613 D(x): 0.7196 D(G(z)): 0.2439 / 0.0610
[2/25][52/75] Loss_D: 0.7519 Loss_G: 3.3164 D(x): 0.7444 D(G(z)): 0.2915 / 0.0656
[2/25][53/75] Loss_D: 0.9237 Loss_G: 3.3085 D(x): 0.6959 D(G(z)): 0.3450 / 0.0620
[2/25][54/75] Loss_D: 0.8445 Loss_G: 2.4389 D(x): 0.6601 D(G(z)): 0.2461 / 0.1326
[2/25][55/75] Loss_D: 0.8128 Loss_G: 5.2308 D(x): 0.8561 D(G(z)): 0.4233 / 0.0110
[2/25][56/75] Loss_D: 0.8791 Loss_G: 2.2688 D(x): 0.5525 D(G(z)): 0.0898 / 0.1697
[2/25][57/75] Loss_D: 0.9041 Loss_G: 6.0567 D(x): 0.9320 D(G(z)): 0.5025 / 0.0059
[2/25][58/75] Loss_D: 0.7510 Loss_G: 3.1778 D(x): 0.5999 D(G(z)): 0.0584 / 0.0744
[2/25][59/75] Loss_D: 0.9600 Loss_G: 3.9917 D(x): 0.8090 D(G(z)): 0.4460 / 0.0331
[2/25][60/75] Loss_D: 0.8753 Loss_G: 4.5958 D(x): 0.7140 D(G(z)): 0.3066 / 0.0165
[2/25][61/75] Loss_D: 0.8544 Loss_G: 1.7734 D(x): 0.5903 D(G(z)): 0.1236 / 0.2196
[2/25][62/75] Lo

[4/25][1/75] Loss_D: 0.8060 Loss_G: 2.6134 D(x): 0.7361 D(G(z)): 0.3012 / 0.0968
[4/25][2/75] Loss_D: 1.1385 Loss_G: 4.1726 D(x): 0.7063 D(G(z)): 0.4733 / 0.0238
[4/25][3/75] Loss_D: 1.0231 Loss_G: 1.9370 D(x): 0.4963 D(G(z)): 0.0947 / 0.1762
[4/25][4/75] Loss_D: 0.9008 Loss_G: 4.9661 D(x): 0.8529 D(G(z)): 0.4850 / 0.0092
[4/25][5/75] Loss_D: 0.6923 Loss_G: 2.8131 D(x): 0.5683 D(G(z)): 0.0359 / 0.0793
[4/25][6/75] Loss_D: 0.6081 Loss_G: 3.7522 D(x): 0.8450 D(G(z)): 0.3219 / 0.0310
[4/25][7/75] Loss_D: 0.3050 Loss_G: 3.8456 D(x): 0.8261 D(G(z)): 0.0915 / 0.0319
[4/25][8/75] Loss_D: 0.4721 Loss_G: 2.8182 D(x): 0.8141 D(G(z)): 0.1111 / 0.0824
[4/25][9/75] Loss_D: 0.7865 Loss_G: 4.6474 D(x): 0.8256 D(G(z)): 0.3790 / 0.0146
[4/25][10/75] Loss_D: 0.8068 Loss_G: 1.9577 D(x): 0.5563 D(G(z)): 0.0770 / 0.2010
[4/25][11/75] Loss_D: 0.8823 Loss_G: 4.6288 D(x): 0.8917 D(G(z)): 0.4770 / 0.0150
[4/25][12/75] Loss_D: 0.9538 Loss_G: 2.0641 D(x): 0.5194 D(G(z)): 0.0792 / 0.1785
[4/25][13/75] Loss_D: 0.9

[5/25][27/75] Loss_D: 0.6497 Loss_G: 3.5922 D(x): 0.6468 D(G(z)): 0.0423 / 0.0450
[5/25][28/75] Loss_D: 0.5977 Loss_G: 4.8733 D(x): 0.8956 D(G(z)): 0.3476 / 0.0123
[5/25][29/75] Loss_D: 0.4799 Loss_G: 3.9454 D(x): 0.7562 D(G(z)): 0.1388 / 0.0303
[5/25][30/75] Loss_D: 0.4628 Loss_G: 3.5930 D(x): 0.8237 D(G(z)): 0.1903 / 0.0393
[5/25][31/75] Loss_D: 0.5878 Loss_G: 4.9661 D(x): 0.8444 D(G(z)): 0.2905 / 0.0105
[5/25][32/75] Loss_D: 0.6611 Loss_G: 3.8575 D(x): 0.6910 D(G(z)): 0.1744 / 0.0319
[5/25][33/75] Loss_D: 0.3367 Loss_G: 4.1907 D(x): 0.8776 D(G(z)): 0.1583 / 0.0215
[5/25][34/75] Loss_D: 0.5012 Loss_G: 5.3218 D(x): 0.8512 D(G(z)): 0.2474 / 0.0075
[5/25][35/75] Loss_D: 0.4480 Loss_G: 3.7665 D(x): 0.7368 D(G(z)): 0.0768 / 0.0319
[5/25][36/75] Loss_D: 0.6085 Loss_G: 5.7716 D(x): 0.8867 D(G(z)): 0.3474 / 0.0052
[5/25][37/75] Loss_D: 0.5624 Loss_G: 3.4372 D(x): 0.6795 D(G(z)): 0.0834 / 0.0452
[5/25][38/75] Loss_D: 0.7033 Loss_G: 6.9234 D(x): 0.9304 D(G(z)): 0.4258 / 0.0017
[5/25][39/75] Lo

...

[12/25][4/75] Loss_D: 0.4551 Loss_G: 5.5003 D(x): 0.8729 D(G(z)): 0.2395 / 0.0065
[12/25][5/75] Loss_D: 0.2721 Loss_G: 4.8151 D(x): 0.8536 D(G(z)): 0.0867 / 0.0162
[12/25][6/75] Loss_D: 0.4463 Loss_G: 4.6314 D(x): 0.8516 D(G(z)): 0.1968 / 0.0172
[12/25][7/75] Loss_D: 0.4544 Loss_G: 4.1273 D(x): 0.8052 D(G(z)): 0.1616 / 0.0292
[12/25][8/75] Loss_D: 0.5219 Loss_G: 4.5934 D(x): 0.8244 D(G(z)): 0.2277 / 0.0166
[12/25][9/75] Loss_D: 0.3996 Loss_G: 3.9111 D(x): 0.7966 D(G(z)): 0.1136 / 0.0333
[12/25][10/75] Loss_D: 0.3792 Loss_G: 3.8759 D(x): 0.8472 D(G(z)): 0.1604 / 0.0331
[12/25][11/75] Loss_D: 0.2841 Loss_G: 4.8022 D(x): 0.9149 D(G(z)): 0.1631 / 0.0119
[12/25][12/75] Loss_D: 0.4161 Loss_G: 3.1905 D(x): 0.7686 D(G(z)): 0.0798 / 0.0591
[12/25][13/75] Loss_D: 0.6193 Loss_G: 7.0270 D(x): 0.9373 D(G(z)): 0.3819 / 0.0016
[12/25][14/75] Loss_D: 0.8398 Loss_G: 3.6751 D(x): 0.5375 D(G(z)): 0.0134 / 0.0410
[12/25][15/75] Loss_D: 0.3696 Loss_G: 4.3729 D(x): 0.9338 D(G(z)): 0.2361 / 0.0170
[12/25][16

[13/25][28/75] Loss_D: 0.4689 Loss_G: 4.6000 D(x): 0.8428 D(G(z)): 0.2078 / 0.0145
[13/25][29/75] Loss_D: 0.6236 Loss_G: 3.5335 D(x): 0.7285 D(G(z)): 0.1850 / 0.0509
[13/25][30/75] Loss_D: 0.5761 Loss_G: 6.0324 D(x): 0.9040 D(G(z)): 0.3226 / 0.0035
[13/25][31/75] Loss_D: 0.4401 Loss_G: 4.1756 D(x): 0.7223 D(G(z)): 0.0400 / 0.0229
[13/25][32/75] Loss_D: 0.3514 Loss_G: 4.3310 D(x): 0.9122 D(G(z)): 0.1947 / 0.0172
[13/25][33/75] Loss_D: 0.4396 Loss_G: 4.4006 D(x): 0.8370 D(G(z)): 0.1869 / 0.0198
[13/25][34/75] Loss_D: 0.4684 Loss_G: 4.1403 D(x): 0.8074 D(G(z)): 0.1725 / 0.0280
[13/25][35/75] Loss_D: 0.5910 Loss_G: 4.2608 D(x): 0.7897 D(G(z)): 0.2460 / 0.0199
[13/25][36/75] Loss_D: 0.5929 Loss_G: 3.4484 D(x): 0.7313 D(G(z)): 0.1859 / 0.0430
[13/25][37/75] Loss_D: 0.5103 Loss_G: 4.9785 D(x): 0.8601 D(G(z)): 0.2587 / 0.0098
[13/25][38/75] Loss_D: 0.4057 Loss_G: 3.5297 D(x): 0.7499 D(G(z)): 0.0640 / 0.0376
[13/25][39/75] Loss_D: 0.3837 Loss_G: 3.9046 D(x): 0.8607 D(G(z)): 0.1880 / 0.0283
[13/

[14/25][52/75] Loss_D: 0.5847 Loss_G: 7.9017 D(x): 0.9496 D(G(z)): 0.3693 / 0.0007
[14/25][53/75] Loss_D: 1.0989 Loss_G: 2.6778 D(x): 0.4343 D(G(z)): 0.0094 / 0.1311
[14/25][54/75] Loss_D: 0.7694 Loss_G: 5.8901 D(x): 0.9276 D(G(z)): 0.4292 / 0.0059
[14/25][55/75] Loss_D: 0.2980 Loss_G: 4.7311 D(x): 0.8091 D(G(z)): 0.0558 / 0.0170
[14/25][56/75] Loss_D: 0.3678 Loss_G: 4.0157 D(x): 0.8631 D(G(z)): 0.1620 / 0.0298
[14/25][57/75] Loss_D: 0.6910 Loss_G: 4.8436 D(x): 0.8081 D(G(z)): 0.3257 / 0.0129
[14/25][58/75] Loss_D: 0.8224 Loss_G: 2.8720 D(x): 0.6136 D(G(z)): 0.1799 / 0.0880
[14/25][59/75] Loss_D: 0.9275 Loss_G: 5.2281 D(x): 0.7918 D(G(z)): 0.4116 / 0.0102
[14/25][60/75] Loss_D: 0.7330 Loss_G: 2.6584 D(x): 0.6175 D(G(z)): 0.0793 / 0.1043
[14/25][61/75] Loss_D: 0.5321 Loss_G: 5.1947 D(x): 0.9214 D(G(z)): 0.3181 / 0.0108
[14/25][62/75] Loss_D: 0.6514 Loss_G: 2.8525 D(x): 0.6551 D(G(z)): 0.0624 / 0.0956
[14/25][63/75] Loss_D: 0.8275 Loss_G: 5.3278 D(x): 0.8727 D(G(z)): 0.3982 / 0.0107
[14/

[16/25][1/75] Loss_D: 0.2461 Loss_G: 4.1019 D(x): 0.8693 D(G(z)): 0.0741 / 0.0267
[16/25][2/75] Loss_D: 0.4965 Loss_G: 3.8493 D(x): 0.8276 D(G(z)): 0.2136 / 0.0363
[16/25][3/75] Loss_D: 0.6978 Loss_G: 3.0662 D(x): 0.6900 D(G(z)): 0.2192 / 0.0748
[16/25][4/75] Loss_D: 0.6689 Loss_G: 5.2654 D(x): 0.8623 D(G(z)): 0.3432 / 0.0106
[16/25][5/75] Loss_D: 0.5066 Loss_G: 3.4293 D(x): 0.6920 D(G(z)): 0.0556 / 0.0516
[16/25][6/75] Loss_D: 0.4721 Loss_G: 3.9848 D(x): 0.8697 D(G(z)): 0.2433 / 0.0281
[16/25][7/75] Loss_D: 0.5402 Loss_G: 5.0711 D(x): 0.8303 D(G(z)): 0.2312 / 0.0107
[16/25][8/75] Loss_D: 0.3935 Loss_G: 3.6938 D(x): 0.7821 D(G(z)): 0.0748 / 0.0379
[16/25][9/75] Loss_D: 0.4224 Loss_G: 4.5124 D(x): 0.8907 D(G(z)): 0.2391 / 0.0159
[16/25][10/75] Loss_D: 0.3275 Loss_G: 3.7423 D(x): 0.8130 D(G(z)): 0.0868 / 0.0339
[16/25][11/75] Loss_D: 0.3377 Loss_G: 4.4494 D(x): 0.9034 D(G(z)): 0.1816 / 0.0186
[16/25][12/75] Loss_D: 0.3441 Loss_G: 3.9398 D(x): 0.8343 D(G(z)): 0.1201 / 0.0284
[16/25][13/75

[17/25][25/75] Loss_D: 0.3237 Loss_G: 4.5976 D(x): 0.8727 D(G(z)): 0.1427 / 0.0166
[17/25][26/75] Loss_D: 0.3722 Loss_G: 4.9939 D(x): 0.8772 D(G(z)): 0.1618 / 0.0105
[17/25][27/75] Loss_D: 0.5038 Loss_G: 3.1292 D(x): 0.7279 D(G(z)): 0.1122 / 0.0787
[17/25][28/75] Loss_D: 0.7640 Loss_G: 7.4355 D(x): 0.9272 D(G(z)): 0.4285 / 0.0014
[17/25][29/75] Loss_D: 0.9801 Loss_G: 3.2183 D(x): 0.5014 D(G(z)): 0.0135 / 0.0738
[17/25][30/75] Loss_D: 0.6023 Loss_G: 5.3233 D(x): 0.9291 D(G(z)): 0.3525 / 0.0115
[17/25][31/75] Loss_D: 0.4999 Loss_G: 3.6675 D(x): 0.7434 D(G(z)): 0.1174 / 0.0385
[17/25][32/75] Loss_D: 0.4176 Loss_G: 4.9999 D(x): 0.9171 D(G(z)): 0.2317 / 0.0114
[17/25][33/75] Loss_D: 0.4518 Loss_G: 3.4074 D(x): 0.7670 D(G(z)): 0.0971 / 0.0595
[17/25][34/75] Loss_D: 0.6507 Loss_G: 6.3846 D(x): 0.8829 D(G(z)): 0.3492 / 0.0029
[17/25][35/75] Loss_D: 0.5836 Loss_G: 3.5536 D(x): 0.6645 D(G(z)): 0.0264 / 0.0604
[17/25][36/75] Loss_D: 0.4691 Loss_G: 4.5280 D(x): 0.8776 D(G(z)): 0.2419 / 0.0161
[17/

## Acknowledgements & Resources
- Much of the code from this notebook is modified from [PyTorch Examples – Deep Convolution Generative Adversarial Networks](https://github.com/pytorch/examples/tree/master/dcgan).
- [Unsupervised Representation Learning with Deep Convolutional Generative Adversarial Networks](https://arxiv.org/pdf/1511.06434.pdf)